In [ ]:
# 텍스트 분석과 챗봇 -> 비번 Bigdata!
# 말뭉치 등의 데이터 : https://konlpy.org/ko/latest/data/#corpora

In [17]:
### 형태소 분석 라이브러리 (koNLPy)       
# pip install konlpy

from konlpy.tag import Twitter, Okt  # 트위터 형태소 분석기 (Twitter -> Okt로 바뀜)

twitter = Okt()
malist = twitter.pos('아니욬ㅋㅋ', norm=True, stem=True)    #  Kkma와 Twitter만 제대로 분석함
        # pos() : POS tagger
        # :param norm: If True, normalize tokens.   # 잘못된 문법을 맞게 변환
        # :param stem: If True, stem tokens.        # (특히 동사를) 원형으로 변환
        # :param join: If True, returns joined sets of morph and tag.
print(malist)

[('아니다', 'Adjective'), ('ㅋㅋ', 'KoreanParticle')]


In [30]:
### Twitter를 이용한 단어 출현 빈도 분석(카운팅)
from konlpy.tag import Twitter, Okt

# (1) 파일을 열고 글자를 출력하기 
fp = open("data/toji1.txt", "r", encoding="utf-8")
text = fp.read()

# (2) 텍스트를 한 줄씩 처리하기
twitter = Okt()
word_dic = {}
lines = text.split('\n')
for line in lines :
    malist = twitter.pos(line)
    for word in malist :
        # 필요한 형태소 추출하기
        if word[1] == 'Noun' and len(word[0]) > 1 :    # 단어가 명사라면
            if not (word[0] in word_dic) :  # word_dic에 포함되어있지 않은 경우 키값으로 추가
                word_dic[word[0]] = 0
            word_dic[word[0]] += 1  # 명사 카운팅

## 정렬을 통해 가장 많이 사용된 명사 출력
keys = sorted(word_dic.items(), key=lambda x : x[1], reverse=True)  # key, value 형태의 튜플 반환
for word, count in keys[:50] :
    print(f"{word}({count})", end=' ')



용이(344) 평산(320) 사람(315) 소리(272) 얼굴(255) 생각(242) 치수(228) 강청댁(198) 서방(185) 어디(164) 마을(149) 칠성(136) 포수(136) 양반(132) 아이(131) 서희(131) 하나(123) 윤씨(122) 참판(120) 월선(117) 다시(112) 부인(112) 마음(110) 보고(109) 그것(107) 마는(106) 모양(101) 구마(101) 우리(101) 여자(101) 의원(97) 바람(95) 세상(91) 무슨(90) 머리(88) 순이(87) 자식(86) 삼수(86) 일이(79) 구천이(78) 서울(78) 기이(77) 애기(77) 윤보(76) 계집(75) 가지(74) 김훈(74) 누가(73) 목소리(72) 준구(71) 

In [43]:
## Word2Vec를 통해 문장을 벡터로 변환하기
from konlpy.tag import Okt
from gensim.models import word2vec

fp = open("data/toji1.txt", "r", encoding="utf-8")
text = fp.read()

twitter = Okt()
results = []

lines = text.split('\n')    # 한 줄씩 처리

# 형태소 분석하기
for line in lines :
    malist = twitter.pos(line, norm=True, stem=True)    # 단어 기본형 사용
    r = []
    for word in malist :
        if not word[1] in ['Josa', 'Eomi', 'Punctuation'] :   # 조사, 어미, 구두점 제외
            r.append(word[0])
    rl = (' '.join(r)).strip()      # 리스트 r의 각 요소를 공백으로 구분하기
    results.append(rl)

# 파일로 출력
toji_file = 'data/toji.clean'
with open(toji_file, 'w', encoding='utf-8') as fp :
    fp.write('\n'.join(results))    # 리스트 각 요소를 \n으로 구분하기
    
# Word2vec 모델 만들기
data = word2vec.LineSentence(toji_file)
model = word2vec.Word2Vec(data, vector_size=200, window=10, hs=1, min_count=2, sg=1)
model.save("data/toji.model")
print("ok")        

ok


In [57]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('data/toji.model')
model.wv.most_similar(positive=['종'])


[('하인', 0.8534995317459106),
 ('최씨', 0.8040891885757446),
 ('계집', 0.7953749895095825),
 ('천하다', 0.7944691181182861),
 ('구걸', 0.793971061706543),
 ('문전', 0.7917537689208984),
 ('서성대다', 0.790041446685791),
 ('런', 0.7880151867866516),
 ('당주', 0.7851266860961914),
 ('내력', 0.7824681997299194)]

In [ ]:
### wiki.text
from konlpy.tag import Twitter, Okt
from gensim.models import Word2Vec

## 파일 열기
read_file = open('data/wiki.txt', 'r', encoding='utf-8')
wiki_file = 'wiki.clean'
write_file = open('data/'+wiki_file, 'w', encoding='utf-8')

## 형태소 분석
twitter = Okt()
i = 0
## 텍스트 한 줄씩 처리
while True :
    line = read_file.readline()     # 한 줄씩 읽기 (포인터를 자동으로 옮겨가면서 읽음)
    if not line : break             # line이 null값일 때 break
    if i % 20000 == 0 :
        print('current -' + str(i))
    i += 1
    
    # 형태소 분석
    malist = twitter.pos(line, norm=True, stem=True)
    
    # 필요한 단어만 추출해서 담기 (필터링)
    r = []
    for word in malist :
        if not word[1] in ['Josa', 'Eomi', 'Punctuation'] : # 조사, 어미, 구두점 제외
            write_file.write(word[0]+" ")
            
write_file.close()
    

In [4]:
from gensim.models import word2vec
data = word2vec.Text8Corpus("data/wiki.clean")
model = word2vec.Word2Vec(data, vector_size=100)
model.save("data/wiki.model")
print("ok")

ok


In [1]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('data/wiki.model')

In [13]:
print(model.wv.most_similar(positive=['Python', '파이썬'])[0:3])
print(model.wv.most_similar(positive=['아빠', '여성'], negative=['남성'])[0])
print(model.wv.most_similar(positive=['오른쪽', '남자'], negative=['왼쪽'])[0]) 
print(model.wv.most_similar(positive=['서울', '일본'], negative=['한국'])[0])   # 수도 뽑기


[('Java', 0.899293839931488), ('Perl', 0.8899776935577393), ('Tcl', 0.8887758255004883)]
('엄마', 0.8427087068557739)
('여자', 0.7638366222381592)
('도쿄', 0.683591902256012)


In [16]:
len(model.wv['고양이'])     # model파일을 추출할 때 벡터사이즈=100으로 설정했기 때문에 모두 동일


100